In [ ]:
import os
import faiss
from Libraries import A0_Widgets, A1_Define, A2_MyUtils
from Libraries import B1_ExtractData, B2_MergeData, B3_GetStructures
from Libraries import B4_ChunkMaster, B5_ChunkFlex, B6_ChunkFixed

In [ ]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
widgets_list = A0_Widgets.create_name_form()

In [ ]:
config = A1_Define.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
extracted_path = config["extracted_path"]
merged_path = config["merged_path"]
struct_path = config["struct_path"]
chunks_base = config["chunks_base"]
chunks_segment = config["chunks_segment"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

SEARCH_ENGINE = faiss.IndexFlatIP

In [ ]:
Contents = LEVEL_VALUES[-1] if LEVEL_VALUES else None
print(Contents)

### Extract Data

In [ ]:
exceptions_path = "../tests/ex.exceptions.json"
markers_path = "../tests/ex.markers.json"
status_path = "../tests/ex.status.json"

In [ ]:
extracted_data = B1_ExtractData.extractData(dcmt_path, exceptions_path, markers_path, status_path)
A2_MyUtils.write_json(extracted_data, extracted_path, indent=1)

In [ ]:
merged_data = B2_MergeData.mergeLinesToParagraphs(extracted_data)
A2_MyUtils.write_json(merged_data, merged_path, indent=1)

### Get Struct

In [ ]:
analyzer = B3_GetStructures.StructureAnalyzer(merged_path, verbose=True)

In [ ]:
def getMarkers():
    markers = analyzer.extract_markers()
    return markers

In [ ]:
def getStructures(markers):
    structures = analyzer.build_structures(markers)
    print(A2_MyUtils.jsonl_convert(structures))
    return structures

In [ ]:
def getDedup(structures):
    dedup = analyzer.deduplicate(structures)
    print(A2_MyUtils.jsonl_convert(dedup))
    return dedup

In [ ]:
def getTop(dedup):
    top = analyzer.select_top(dedup)
    topext = analyzer.extend_top(top, dedup)
    print(A2_MyUtils.json_convert(topext, pretty=True))
    return topext

In [ ]:
markers = getMarkers()
structures = getStructures(markers)
dedup = getDedup(structures)
topext = getTop(dedup)

In [ ]:
A2_MyUtils.write_json(topext, struct_path, indent=2)

### Chunks

In [ ]:
builder = B4_ChunkMaster.ChunkBuilder(
    struct_path=struct_path,
    merged_path=merged_path,
    output_path="Data_HNMU_Regulations_Chunks.json"
)

chunks = builder.build()
A2_MyUtils.write_json(chunks, chunks_base, indent=2)